In [1]:
import torch
print(torch.cuda.is_available())         # should return True
print(torch.cuda.get_device_name(0))     # should say "NVIDIA GeForce RTX 3060"


True
NVIDIA GeForce RTX 3060


Imports 

In [2]:
import os

# For data manipulation and analysis 
import pandas as pd

# For numerical operations and array handling 
import numpy as np

# For creating plots and visualisations
import matplotlib.pylab as plt

# For advanced data visualisations 
import seaborn as sns

# For file pattern matching 
import glob as glob

# For audio processing 
import librosa

# For displaying audio data visually 
import librosa.display

# For playing audio directly in notebooks 
import IPython.display as ipd

import wave



Loading dataset 

In [10]:
base_dir = os.path.join(os.getcwd(), 'datasets', 'voice-based-id-recognition')
diffPhrase = os.path.join(base_dir,'differentPhrase')
samePhrase = os.path.join(base_dir,'samePhrase')

Checking Dataset Location existance

In [11]:
# Check if the training directory exists
if os.path.exists(diffPhrase):
    print("differentPhrase Directory: ")
    print(os.listdir(diffPhrase))
else:
    print("Directory does not exist:", diffPhrase) 
   

# Check if the testing directory exists
if os.path.exists(samePhrase):
    
    print("samePhrase Directory: ")
    print(os.listdir(samePhrase))
else:
    print("Directory does not exist:", samePhrase)

differentPhrase Directory: 
['1', '10', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']
samePhrase Directory: 
['1', 

Listen to audio files

In [13]:


file_path = r"C:\Users\Aadil\Desktop\Voice-Auth-DF-System\ml-models\datasets\voice-based-id-recognition\differentPhrase\1\1-11.wav"

with wave.open(file_path, 'rb') as wav_file:
    num_frames = wav_file.getnframes()
    sample_rate = wav_file.getframerate()
    sample_width = wav_file.getsampwidth()
    channels = wav_file.getnchannels()
    audio_data = wav_file.readframes(num_frames)

# Convert raw bytes to numpy array
audio_np = np.frombuffer(audio_data, dtype=np.int16)

# If stereo, reshape
if channels == 2:
    audio_np = audio_np.reshape(-1, 2)

ipd.Audio(audio_np, rate=sample_rate)


